In [12]:
import numpy as np
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50, VGG16, InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [13]:
# Cargar el dataset
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
X = lfw_people.images
y = lfw_people.target
X.shape, y.shape

((1288, 50, 37), (1288,))

In [14]:
# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1030, 50, 37), (258, 50, 37), (1030,), (258,))

In [15]:
# Escalar las imágenes
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X.shape[1] * X.shape[2]))
X_test = scaler.transform(X_test.reshape(-1, X.shape[1] * X.shape[2]))
X_train.shape, X_test.shape

((1030, 1850), (258, 1850))

In [16]:
# Redimensionar las imágenes para que tengan la forma adecuada para los modelos pre-entrenados
X_train = np.repeat(X_train.reshape(-1, X.shape[1], X.shape[2], 1), 3, axis=3)
X_test = np.repeat(X_test.reshape(-1, X.shape[1], X.shape[2], 1), 3, axis=3)
X_train.shape, X_test.shape

((1030, 50, 37, 3), (258, 50, 37, 3))

In [17]:
# Convertir las etiquetas a one-hot encoding
num_classes = len(np.unique(y))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
num_classes, y_train.shape, y_test.shape

(7, (1030, 7), (258, 7))

In [18]:
# Cargar y configurar el modelo ResNet50
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(X.shape[1], X.shape[2], 3))
for layer in resnet_model.layers:
    layer.trainable = False
x = resnet_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
resnet_model = Model(inputs=resnet_model.input, outputs=predictions)

# Compilar y entrenar el modelo ResNet50
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
resnet_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

#Evaluamos
test_loss, test_acc = resnet_model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

Epoch 1/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 11s 168ms/step - accuracy: 0.3680 - loss: 1.7651 - val_accuracy: 0.4419 - val_loss: 1.5771
Epoch 2/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 113ms/step - accuracy: 0.5274 - loss: 1.4041 - val_accuracy: 0.4767 - val_loss: 1.4285
Epoch 3/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 113ms/step - accuracy: 0.5683 - loss: 1.2382 - val_accuracy: 0.5736 - val_loss: 1.3061
Epoch 4/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 115ms/step - accuracy: 0.6246 - loss: 1.0936 - val_accuracy: 0.5853 - val_loss: 1.1955
Epoch 5/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 116ms/step - accuracy: 0.6908 - loss: 0.9971 - val_accuracy: 0.5853 - val_loss: 1.2748
Epoch 6/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 114ms/step - accuracy: 0.6663 - loss: 0.9482 - val_accuracy: 0.6395 - val_loss: 1.1287
Epoch 7/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 115ms/step - accuracy: 0.7179 - loss: 0.8588 - val_accuracy: 0.6085 - val_loss: 1.2177
Epoch 8/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 126ms/step - accuracy: 0.7411 - loss: 0.8026 - val_accuracy: 0

In [19]:
# Cargar y configurar el modelo VGG16
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(X.shape[1], X.shape[2], 3))
for layer in vgg_model.layers:
    layer.trainable = False
x = vgg_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
vgg_model = Model(inputs=vgg_model.input, outputs=predictions)

# Compilar y entrenar el modelo VGG16
vgg_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
vgg_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluamos

test_loss, test_acc = vgg_model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

Epoch 1/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 5s 130ms/step - accuracy: 0.4692 - loss: 1.5783 - val_accuracy: 0.5969 - val_loss: 1.1878
Epoch 2/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 123ms/step - accuracy: 0.6244 - loss: 1.0381 - val_accuracy: 0.6202 - val_loss: 1.0150
Epoch 3/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 122ms/step - accuracy: 0.7365 - loss: 0.8176 - val_accuracy: 0.6589 - val_loss: 0.9351
Epoch 4/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 121ms/step - accuracy: 0.7725 - loss: 0.7110 - val_accuracy: 0.6860 - val_loss: 0.8658
Epoch 5/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 126ms/step - accuracy: 0.8184 - loss: 0.5535 - val_accuracy: 0.6473 - val_loss: 1.0068
Epoch 6/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 128ms/step - accuracy: 0.8367 - loss: 0.5008 - val_accuracy: 0.6938 - val_loss: 0.8632
Epoch 7/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 123ms/step - accuracy: 0.8564 - loss: 0.4310 - val_accuracy: 0.6899 - val_loss: 0.8501
Epoch 8/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 122ms/step - accuracy: 0.9106 - loss: 0.3545 - val_accuracy: 0.